## Profilowanie CPU

- dekorowanie funkcji z użyciem funkcji time.time()
- cProfile i pstats
- line_profiler
- %timeit
- dis

In [ ]:
# cProfiler wyświetla informacje o czaie wywołąnia fukncji

# profiling/ex1.py
def has_item(collection, item):
    found = False
    for i in collection:
        if item == i:
            found = True
    return found


#collection = range(100000000)
#print has_item(collection, 100000000)




# profiling/ex1a.py
def has_item(collection, item):
    return item in collection


#collection = range(100000000)
#print has_item(collection, 100000000)



# profiling/ex1.sh
# python -m cProfile ex1.py

# python -m cProfile ex1a.py

In [ ]:
# profiling/ex2.py
# line profiler wyświetla czasy wykonania poszczagólnych linii
# do wyprofilowania funkcji, należy ją udekorować dekoratorem @profile i uruchomić przez kernprof

@profile  # ten dekorator nie jest z nikąd importowany
def has_item(collection, item):
    found = False
    for i in collection:
        if item == i:
            found = True
    return found


#collection = range(100000000)
#print has_item(collection, 100000000)


# profiling/ex2.sh
#kernprof -l -v ex2.py

In [ ]:
# %timeit w IPythonie pozwala na profilowanie w sesji interaktywnego interpretera

%timeit has_item(range(10000000), 10000000)

In [ ]:
# dis słuzy do deasemblacji kodu pythonowego na instrukcje dla maszyny wirtualnej
import dis


def f():
    x = 1
    y = []
    y.append(x)
    print y
    return x ** 2


dis.dis(f)

## Profilowanie Pamięci

- memory_profiler
- %memit

In [ ]:
# profiling/ex3.py
# moduł psutil przyspiesza działąnie mempry_profilera

def has_item(collection, item):
    found = False
    for i in collection:
        if item == i:
            found = True
    return found


@profile  # tak jak w przypadku linear_profiler dekorujemy funkcję nieimportowanym dekoratorem
def main():
    collection = range(100000000)
    print has_item(collection, 100000000)


#main()


# profiling/ex3.sh
#python -m memory_profiler ex3.py

In [ ]:
# %memit działą w IPythonie
%load_ext memory_profiler
%memit [0] * 100000000  # nienajgorzej

#%memit range(100000000)  # 3GB
%memit

# Generalnie tytpy podstawowe (int, float, string) jako typyt pythonowe są "cięższe"
# niż ich odpowiedniki w C

# moduł array przechowuje typy podstawowe bez opakowywania ich w obiektowość Pythonową -
# do czasu aż nie zaczniemy ich stamtąd wybierać - wtedy liczby będą opakowane w typy Pythonowe
# w Cythonie nie ma narzutu ramu przy wybieraniu obiektów z array

print "array"
import array
%memit array.array('l', xrange(100000000))  # nieźle, 700 MB zamiast 3GB

print help(array)  # opisane jaka litera to jaki typ

## Efektywne wykorzystanie typów wbudowanych i zasobów

### Lista i Krotka
lista jest alokowana z pewnym zapasem (dodatkowe puste pola), ale jeżeli zostanie zapełńiona, to zostaje zaalokowana nowa pamięć na jeszcze większą listę a oryginalna lista zostaje skopiowana z nowe miejsce. Krotka nie alokuje dodatkowego miejsca na nowe elementy (bo nie ma po co), dlatego w przypadku, kiedy lista ma byc niezmienna oszczedniej jest użyć krotki zamist listy. Złożoność obliczeniowa dostępu do doługości listy/krotki to O(1)

### Słownik i zbiór

Używają tablicy haszujacych do wsadzania/wyjmowania obiektów. Obiekt musi mieć zaimplementowane metody \_\_hash\_\_ i \_\_eq\_\_ żeby moża go było wsadzić do zbioru/słownika. Dzięki temu złozónosć obliczeniowa dodawania/usuwania ze zbioru/słownika to O(1) natomiast nie ma mozliwosći umieszczenia kiklu elementów o tym samym haszu. Dostęp do ilości elementów wynosi O(1)

### Generatory i Iteratory
Są przydatne podczas iteracji, szczególnie, jeżeli element po którym jest iterowane miałby być użytyt tylko raz (np. for i in range(X)).

Pozwalają zaoszczędzić czas i pamięć na tworzenie listy (Generatory/Iteratory, przechowują tylko stan a nie elementy sekwencji) a ponadto umożliwiają generowanie nieskończonych sekwencji i przerywanie wykonia aż do następnego wybudzenia.

### Własne klasy

Jeżeli mamy własną klasę, która ma mieć określony zestaw atrybutów a instancji tej klasy ma być bardzo dużo, można użīc obiektu namedtuple z modułu collections lub we własnej klasie zdefiniować atrybut \_\_slots\_\_

In [ ]:
from collections import namedtuple

A = namedtuple('A', 'a b c')  # zamiast stringa, może byc lista z nazwami atrybutów
print A
a = A(1, 2, 3)
print a, a[0], a.a  # do atrybutów namedtuple można odwoływać się przez indeks albo nazwę

print 50 * '='

class Slotted(object):
    __slots__ = ['a', 'b', 'c']

s = Slotted()
s.a = 1
s.b = 1
s.c = 1

print s, s.a, s.b, s.c
s.d = 44  # AttributeError - nie możńa dodawać atrybutów

# Inne metody optymalizacji

## Uruchomienie kodu przez PyPy

Implementacją, która często jest polecana przy problemach z wydajnością, jest impolementacja w Pythonie (PyPy). Dzięki kompilacji JIT (just in time - podczas wykonania kodu) rezultaty są często porównywalne do kodu napisanego w C

## Kompilacja kodu i rozszerzenia w C

- Cython
- C

Do optymalizacji wąskich gardeł można wykorzystać kod napisany w Cythonie (język pośredni pomiędzy C i Pythonem), który następnie jest kompilowany do biblioteki .so, którą można zaimportować i używać bezposrednio w kodzie Pyhonowym.

Do napisania modułu bezpośrednio w C konieczne jest dołączenie pliku nagłówkowego Python.h